In [1]:
!pip install llama-index==0.11.11 -q


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install docx2txt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## 2.2 데이터 로딩

### 2.2.1 데이터 리더

In [1]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("sample_docs").load_data()

Ignoring wrong pointing object 6 0 (offset 0)


In [2]:
# Document 리스트의 각 요소에 접근하여 내용 출력
for i, document in enumerate(documents):
    print(f"Document {i+1}:")
    print(document.text)
    print("\n")

Document 1:
This is the content of file 1.


Document 2:
This is the content of file 2.


Document 3:
This is the content of ﬁle 3. 




In [3]:
documents = SimpleDirectoryReader(
    "sample_docs", recursive=True).load_data()

Ignoring wrong pointing object 6 0 (offset 0)


In [4]:
# Document 리스트의 각 요소에 접근하여 내용 출력
for i, document in enumerate(documents):
    print(f"Document {i+1}:")
    print(document.text)
    print("\n")


Document 1:
This is the content of file 1.


Document 2:
This is the content of file 2.


Document 3:
This is the content of ﬁle 3. 


Document 4:
This is the content of file 4.


Document 5:
This is the content of file 5.




In [5]:
documents = SimpleDirectoryReader(
    "sample_docs",
    required_exts=[".txt", ".pdf"],
    recursive=True).load_data()

Ignoring wrong pointing object 6 0 (offset 0)


In [6]:
# Document 리스트의 각 요소에 접근하여 내용 출력
for i, document in enumerate(documents):
    print(f"Document {i+1}:")
    print(document.text)
    print("\n")


Document 1:
This is the content of file 1.


Document 2:
This is the content of ﬁle 3. 


Document 3:
This is the content of file 4.




In [7]:
# document 리스트의 각 요소에 접근하여 메타데이터를 출력
for i, document in enumerate(documents):
    print(f"Document {i+1} metadata:")
    print(document.metadata)
    print("\n")

Document 1 metadata:
{'file_path': '/external/llama-index-tutorial/ch02/sample_docs/file1.txt', 'file_name': 'file1.txt', 'file_type': 'text/plain', 'file_size': 30, 'creation_date': '2025-07-04', 'last_modified_date': '2025-07-04'}


Document 2 metadata:
{'page_label': '1', 'file_name': 'file3.pdf', 'file_path': '/external/llama-index-tutorial/ch02/sample_docs/file3.pdf', 'file_type': 'application/pdf', 'file_size': 9283, 'creation_date': '2025-07-04', 'last_modified_date': '2025-07-04'}


Document 3 metadata:
{'file_path': '/external/llama-index-tutorial/ch02/sample_docs/sub_sample_docs/file4.txt', 'file_name': 'file4.txt', 'file_type': 'text/plain', 'file_size': 30, 'creation_date': '2025-07-04', 'last_modified_date': '2025-07-04'}




### 2.2.2 데이터 커넥터

In [19]:
!pip install llama-index-readers-database==0.3.0 -q


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [21]:
!pip install pymysql


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [25]:
from sqlalchemy import create_engine
from llama_index.readers.database import DatabaseReader

# MySQL 연결 정보 직접 입력
scheme = "mysql+pymysql"
host = "localhost"
password = "!23"
port = "3306"
user = "root"
dbname = "test_db"

connection_string = f"{scheme}://{user}:{password}@{host}:{port}/{dbname}"
engine = create_engine(connection_string)
reader = DatabaseReader(sql_database=engine)

# 데이터 로드
query = "SELECT * FROM users"
documents = reader.load_data(query=query)


In [26]:
# documents 리스트의 각 요소 출력
for idx, doc in enumerate(documents):
    print(f"Document {idx + 1}:")
    print(f"ID {doc.id_}")
    print(f"Row {doc.text}")
    print("-" * 50)


Document 1:
ID 26ac5d69-f2a0-4774-b8db-7f44e8ddaf5a
Row id: 1, name: Alice, email: alice@example.com, age: 30
--------------------------------------------------
Document 2:
ID f5d575a4-1170-404b-985a-cefa40909b6a
Row id: 2, name: Bob, email: bob@example.com, age: 25
--------------------------------------------------
Document 3:
ID e98158f4-09ed-4979-9b3f-85e5db629004
Row id: 3, name: Charlie, email: charlie@example.com, age: 35
--------------------------------------------------


## 2.3 텍스트 분할

### 2.3.1 문서와 노드

In [9]:
from llama_index.core import Document

# 텍스트 데이터를 기반으로 문서 생성
document_text = "영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 박 사장네 집에는 비밀 지하실이 존재하며, 그곳에는 오랫동안 숨어 살던 남자가 있다는 반전이 있습니다. 이 사실을 알게 된 기우네 가족은 예상치 못한 위기를 맞이하게 됩니다. 결국 극한 상황에서 벌어지는 사건으로 인해 비극적인 결말로 이어집니다."
document = Document(text=document_text)

# 메타데이터 추가
document.metadata = {'author': '영화 해설', 'subject': '기생충 줄거리'}

print(document)

Doc ID: 77c3f790-2236-4789-97b3-09ec2ef10841
Text: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 처음에는
평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 박 사장네 집에는 비밀 지하실이 존재하며, 그곳에는
오랫동안 숨어 살던 남자가 있다는 반전이 있습니다. 이 사실을 알게 된 기우네 가족은 예상치 못한 위기를 맞이하게 됩니다.
결국 극한 상황에서 벌어지는 사건으로 인해 비극적인 결말로 이어집니다.


In [10]:
from llama_index.core.node_parser import SimpleNodeParser

parser = SimpleNodeParser(chunk_size=80, chunk_overlap=0)
nodes = parser.get_nodes_from_documents([document])

print("\n생성된 노드들")
for idx, node in enumerate(nodes, start=1):
    node.metadata = {'type': '영화 줄거리', 'genre': '드라마', 'node_id': idx}
    print(f"노드 {idx}: {node.text}")
    print(f"   메타데이터: {node.metadata}")


생성된 노드들
노드 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다.
   메타데이터: {'type': '영화 줄거리', 'genre': '드라마', 'node_id': 1}
노드 2: 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다.
   메타데이터: {'type': '영화 줄거리', 'genre': '드라마', 'node_id': 2}
노드 3: 박 사장네 집에는 비밀 지하실이 존재하며, 그곳에는 오랫동안 숨어 살던 남자가 있다는 반전이 있습니다.
   메타데이터: {'type': '영화 줄거리', 'genre': '드라마', 'node_id': 3}
노드 4: 이 사실을 알게 된 기우네 가족은 예상치 못한 위기를 맞이하게 됩니다.
   메타데이터: {'type': '영화 줄거리', 'genre': '드라마', 'node_id': 4}
노드 5: 결국 극한 상황에서 벌어지는 사건으로 인해 비극적인 결말로 이어집니다.
   메타데이터: {'type': '영화 줄거리', 'genre': '드라마', 'node_id': 5}


In [11]:
from llama_index.core.node_parser import TokenTextSplitter
import tiktoken

# cl100k_base 토크나이저 로드
tokenizer = tiktoken.get_encoding("cl100k_base")

token_splitter = TokenTextSplitter(chunk_size=80, chunk_overlap=0)

chunks = token_splitter.split_text(document_text)

for i, chunk in enumerate(chunks):
    token_count = len(tokenizer.encode(chunk))  # 각 청크의 토큰 개수 계산
    print(f"청크 {i+1}: {chunk}\n[토큰 개수: {token_count}]\n")

print(f"총 생성된 청크 개수: {len(chunks)}")


청크 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 처음에는 평화로워 보이지만, 이들의
[토큰 개수: 77]

청크 2: 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 박 사장네 집에는 비밀 지하실이 존재하며, 그곳에는 오랫동안 숨어 살던 남자가 있다는
[토큰 개수: 78]

청크 3: 반전이 있습니다. 이 사실을 알게 된 기우네 가족은 예상치 못한 위기를 맞이하게 됩니다. 결국 극한 상황에서 벌어지는 사건으로 인해 비극적인 결말로 이어집니다.
[토큰 개수: 80]

총 생성된 청크 개수: 3


In [12]:
from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.llms.openai import OpenAI

# 한글 답변 설정
llm = OpenAI(api_key=os.environ["OPENAI_API_KEY"], model="gpt-4o-mini", system_prompt="반드시 한국어로 답변하세요.")
# Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0, system_prompt="항상 한국어로 답변하세요.")

# 문서 단위 검색을 위한 전체 문서 인덱스 생성
full_doc_index = VectorStoreIndex.from_documents([document], llm=llm)

# NodeParser를 사용하여 문서를 작은 의미 단위(Node)로 분할
# parser = SimpleNodeParser(chunk_size=100, chunk_overlap=0)
# nodes = parser.get_nodes_from_documents([document])

# 노드 단위 검색을 위한 인덱스 생성
node_index = VectorStoreIndex(nodes, llm=llm)

# 검색 비교
query_text = '이 영화의 반전은 무엇인가요?'

## 문서 단위 검색
print("\n문서 단위 검색 결과")
doc_query_engine = full_doc_index.as_query_engine()
doc_response = doc_query_engine.query(query_text)
print(f"문서 검색 응답: {doc_response.response}")
if doc_response.source_nodes:
    for idx, document in enumerate(doc_response.source_nodes, start=1):
        print(f"- 결과 {idx}: {document.node.text}")

## 노드 단위 검색
print("\n노드 단위 검색 결과")
node_query_engine = node_index.as_query_engine()
node_response = node_query_engine.query(query_text)
print(f"노드 검색 응답: {node_response.response}")
if node_response.source_nodes:
    for idx, document in enumerate(node_response.source_nodes, start=1):
        print(f"- 결과 노드 {idx}: {document.node.text}")



문서 단위 검색 결과
문서 검색 응답: 이 영화의 반전은 박 사장네 집에 숨어 살던 남자가 존재한다는 사실입니다.
- 결과 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 박 사장네 집에는 비밀 지하실이 존재하며, 그곳에는 오랫동안 숨어 살던 남자가 있다는 반전이 있습니다. 이 사실을 알게 된 기우네 가족은 예상치 못한 위기를 맞이하게 됩니다. 결국 극한 상황에서 벌어지는 사건으로 인해 비극적인 결말로 이어집니다.

노드 단위 검색 결과
노드 검색 응답: 남자가 박 사장네 집의 비밀 지하실에 숨어 살고 있다는 반전입니다.
- 결과 노드 1: 박 사장네 집에는 비밀 지하실이 존재하며, 그곳에는 오랫동안 숨어 살던 남자가 있다는 반전이 있습니다.
- 결과 노드 2: 결국 극한 상황에서 벌어지는 사건으로 인해 비극적인 결말로 이어집니다.


### 2.3.2 토큰 단위 분할

In [13]:
sample_text = "영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 그러나 이 영화의 반전은 지하실에서 시작됩니다."


token_splitter = TokenTextSplitter(
    chunk_size=50,
    chunk_overlap=10
)

token_chunks = token_splitter.split_text(sample_text)
print("=== 토큰 기반 분할 결과 ===")
for i, chunk in enumerate(token_chunks):
    print(f"Chunk {i + 1}:", chunk.strip(), "\n")



=== 토큰 기반 분할 결과 ===
Chunk 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 

Chunk 2: 취업하면서 벌어지는 이야기입니다. 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 

Chunk 3: 쌓이며 긴장감이 점점 고조됩니다. 그러나 이 영화의 반전은 지하실에서 시작됩니다. 



### 2.3.3 문장 단위 분할

In [14]:
from llama_index.core.node_parser.text.sentence import SentenceSplitter

splitter = SentenceSplitter(chunk_size=50, chunk_overlap=0)

sentence_chunks = splitter.split_text(sample_text)
print("=== 문장 기반 분할 결과 ===")
for i, chunk in enumerate(sentence_chunks):
    print(f"Chunk {i + 1}:", chunk.strip(), "\n")


=== 문장 기반 분할 결과 ===
Chunk 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 

Chunk 2: 벌어지는 이야기입니다. 

Chunk 3: 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 

Chunk 4: 그러나 이 영화의 반전은 지하실에서 시작됩니다. 



### 2.3.4 의미 단위 분할

In [15]:
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding()

splitter = SemanticSplitterNodeParser(
    buffer_size=1,
    breakpoint_percentile_threshold=95,
    embed_model=embed_model
)

chunks = splitter.sentence_splitter(sample_text)
print("=== 의미 기반 분할 결과 ===")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:", chunk.strip(), "\n")


=== 의미 기반 분할 결과 ===
Chunk 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 

Chunk 2: 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 

Chunk 3: 그러나 이 영화의 반전은 지하실에서 시작됩니다. 



In [17]:
!pip install transformers -q
!pip install llama-index-embeddings-huggingface -q


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [18]:
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

splitter = SemanticSplitterNodeParser(
    buffer_size=1,
    breakpoint_percentile_threshold=95,
    embed_model=embed_model
)

chunks = splitter.sentence_splitter(sample_text)
print("=== 의미 기반 분할 결과 ===")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:", chunk.strip(), "\n")

=== 의미 기반 분할 결과 ===
Chunk 1: 영화 '기생충'은 가난한 가족인 기우네가 부유한 박 사장네 집에 하나씩 취업하면서 벌어지는 이야기입니다. 

Chunk 2: 처음에는 평화로워 보이지만, 이들의 거짓말이 쌓이며 긴장감이 점점 고조됩니다. 

Chunk 3: 그러나 이 영화의 반전은 지하실에서 시작됩니다. 



## 2.4 인덱싱

### 2.4.2 벡터 저장소 인덱스